In [1]:
import requests
import json
import pymysql
import time
import datetime
import traceback

In [2]:
#reading in the file with the Apikey and login information. 
f = open('Open_weather_API_login.txt')
line = f.readlines() 
converted_list=[]

for element in line:
    converted_list.append(element.strip())


STATIONS_URI=converted_list[0]
APIKEY=converted_list[1]
host=converted_list[2]
db_name=converted_list[3]
user_name=converted_list[4]
password=converted_list[5]

#cursorType=pymysql.cursors.DictCursor



In [3]:
lat = "53.3498"
lon = "-6.2603"
url = "https://api.openweathermap.org/data/2.5/weather?lat=%s&lon=%s&appid=%s&units=metric" % (lat, lon, APIKEY)

In [4]:
def get_data(url):
    
    #Connecting to the OPen Weather API 
    r=requests.get(url)
    json_obj=json.loads(r.text)
   
    return json_obj

def convert_data(json_obj):   
    #Connecting to the dublinbus DB
    connectionObject = pymysql.connect(host = host,user = user_name,db=db_name, passwd = password,port=3306)
    cursorObject = connectionObject.cursor()

    #Creating table current_weather in dublinbus DB if it doesn't exist.
    sqlQuery= "CREATE TABLE IF NOT EXISTS current_weather(date DATETIME, temp VARCHAR(5), feels_like FLOAT,  wind_speed FLOAT,clouds_all INT ,weather_id INT,weather_main VARCHAR(20),weather_description VARCHAR (45),sunrise DATETIME,sunset DATETIME,icon VARCHAR(10))"
    cursorObject.execute(sqlQuery)
    # parse json data to SQL insert
    for i in enumerate(json_obj):
        date=json_obj.get("dt")
        temp=json_obj.get("main").get("temp", None)
        feels_like=json_obj.get("main").get("feels_like", None)
        wind_speed=json_obj.get("wind").get("speed", None)
        clouds_all=json_obj.get("clouds").get("all", None)
        weather_id=json_obj.get("weather")[0].get("id", None)
        weather_main=json_obj.get("weather")[0].get("main", None)
        weather_description=json_obj.get("weather")[0].get("description", None)
        sunrise = json_obj.get("sys").get("sunrise", None)
        sunset=json_obj.get("sys").get("sunset", None)
        icon=json_obj.get("weather")[0].get("icon", None)

    #converting UTC time into datetime for insertion to database    
    dt=int(date)
    sr=int(sunrise)
    ss=int(sunset)
    date=datetime.datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
    sunrise=datetime.datetime.utcfromtimestamp(sr).strftime('%Y-%m-%d %H:%M:%S')
    sunset=datetime.datetime.utcfromtimestamp(ss).strftime('%Y-%m-%d %H:%M:%S')

    #Inserting values into current_weather table in dublinbus schema.
    cursorObject.execute("INSERT INTO current_weather (date,temp,feels_like,wind_speed, clouds_all, weather_id,weather_main,weather_description,sunrise,sunset,icon) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (date,temp,feels_like,wind_speed, clouds_all, weather_id,weather_main,weather_description,sunrise,sunset,icon))

    
    
    connectionObject.commit()
    connectionObject.close()

def main():

    try:
        json_obj=get_data(url)
        df=convert_data(json_obj)
    except:
        print(traceback.format_exc())

if __name__ == "__main__":
    main()